In [1]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("/root/jupyter_notebooks/pynq-original-dpu-model/dpu.bit")

In [2]:
from pynq import Overlay
from pynq.lib import AxiGPIO
ol = overlay

In [3]:
# GPIO_set
import time

gpio_0_ip = ol.ip_dict['axi_gpio_0']
gpio_out = AxiGPIO(gpio_0_ip).channel1

mask = 0xffffffff
gpio_out.write(0x07,mask) #Blue_led_on + 2*motor_driver_enable

In [4]:
# PWM_set
# utility functions for bit manipulation
def set_bit(value, bit):
    return value | (1 << bit)

def clear_bit(value, bit):
    return value & ~(1 << bit)

def get_bit(value, bit):
    return (value >> bit) & 1

motor_B2 = ol.axi_timer_0
motor_B1 = ol.axi_timer_1
motor_A2 = ol.axi_timer_2
motor_A1 = ol.axi_timer_3

# extract register addresses (will be the same for every Axi Timer)
TCSR0 = ol.ip_dict['axi_timer_0']['registers']['TCSR0']
TCSR1 = ol.ip_dict['axi_timer_0']['registers']['TCSR1']
TCSR0_address = TCSR0['address_offset']
TCSR1_address = TCSR1['address_offset']
TCSR0_register = TCSR0['fields'] # bit_offset for address
TCSR1_register = TCSR1['fields']
TLR0 = ol.ip_dict['axi_timer_0']['registers']['TLR0']
TLR1 = ol.ip_dict['axi_timer_0']['registers']['TLR1']
TLR0_address = TLR0['address_offset']
TLR1_address = TLR1['address_offset']

# create the configuration values for the control register
temp_val_0 = 0
temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
temp_val_0 = set_bit(temp_val_0, TCSR0_register['PWMA0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
temp_val_0 = set_bit(temp_val_0, TCSR0_register['GENT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
temp_val_0 = set_bit(temp_val_0, TCSR0_register['UDT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['UDT1']['bit_offset'])

# set Autoreload (ARHT0 = 1)
temp_val_0 = set_bit(temp_val_0, TCSR0_register['ARHT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['ARHT1']['bit_offset'])

# enable timer (ENT0 = 1)
temp_val_0 = set_bit(temp_val_0, TCSR0_register['ENT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['ENT1']['bit_offset'])

# here you must see "0b1010010110" twice
print(bin(temp_val_0))
print(bin(temp_val_1))


0b1010010110
0b1010010110


In [5]:
# PWM_A_motor_set

def set_motor_A_pwm(duty_cycle, direction):
    _period_ = 20000  # 50Hz, 20ms
    _pulse_ = duty_cycle  # 0-100
    period = int((_period_ & 0x0ffff) * 100)
    pulse = int((_pulse_ & 0x07f) * period / 100)

    print(f"period 20ms: {period}")
    print(f"pulse {duty_cycle}%: {pulse}")
    print(f"direction: {direction}")

    motor_A1.write(TCSR0['address_offset'], temp_val_0)
    motor_A1.write(TCSR1['address_offset'], temp_val_1)
    motor_A1.write(TLR0['address_offset'], period)

    motor_A2.write(TCSR0['address_offset'], temp_val_0)
    motor_A2.write(TCSR1['address_offset'], temp_val_1)
    motor_A2.write(TLR0['address_offset'], period)

    # direction
    if direction == 'forward':
        motor_A1.write(TLR1['address_offset'], pulse)
        motor_A2.write(TLR1['address_offset'], 0)
    elif direction == 'reverse':
        motor_A1.write(TLR1['address_offset'], 0)
        motor_A2.write(TLR1['address_offset'], pulse)
    elif direction == 'coast':
        motor_A1.write(TLR1['address_offset'], 0)
        motor_A2.write(TLR1['address_offset'], 0)
    elif direction == 'break':
        motor_A1.write(TLR1['address_offset'], 100)
        motor_A2.write(TLR1['address_offset'], 100)   
    else:
        print("Invalid direction. Please use 'forward' or 'reverse'or 'coast' or 'break'.")

In [6]:
# PWM_B_motor_set

def set_motor_B_pwm(duty_cycle, direction):
    _period_ = 20000  # 50Hz, 20ms
    _pulse_ = duty_cycle  # 0-100
    period = int((_period_ & 0x0ffff) * 100)
    pulse = int((_pulse_ & 0x07f) * period / 100)

    print(f"period 20ms: {period}")
    print(f"pulse {duty_cycle}%: {pulse}")
    print(f"direction: {direction}")

    motor_B1.write(TCSR0['address_offset'], temp_val_0)
    motor_B1.write(TCSR1['address_offset'], temp_val_1)
    motor_B1.write(TLR0['address_offset'], period)

    motor_B2.write(TCSR0['address_offset'], temp_val_0)
    motor_B2.write(TCSR1['address_offset'], temp_val_1)
    motor_B2.write(TLR0['address_offset'], period)

    # direction
    if direction == 'forward':
        motor_B1.write(TLR1['address_offset'], pulse)
        motor_B2.write(TLR1['address_offset'], 0)
    elif direction == 'reverse':
        motor_B1.write(TLR1['address_offset'], 0)
        motor_B2.write(TLR1['address_offset'], pulse)
    elif direction == 'coast':
        motor_B1.write(TLR1['address_offset'], 0)
        motor_B2.write(TLR1['address_offset'], 0)
    elif direction == 'break':
        motor_B1.write(TLR1['address_offset'], 100)
        motor_B2.write(TLR1['address_offset'], 100)   
    else:
        print("Invalid direction. Please use 'forward' or 'reverse'or 'coast' or 'break'.")

In [7]:
from inputs import devices

# Retrieve the gamepad device
gamepads = devices.gamepads
print(gamepads)
if not gamepads:
    raise ValueError("No gamepad found.")

[inputs.GamePad("/dev/input/by-id/usb-ELECOM_JC-U4113S_DirectInput_Mode-event-joystick")]


In [8]:
#Left(B)-motor_set
def control_motor_based_on_joy_ly(joy_ly):
    # Control motor based on joy_ly value divided into 8 segments
    if 0 <= joy_ly < 32:  # Segment 1
        set_motor_B_pwm(50, 'forward')
    elif 32 <= joy_ly < 64:  # Segment 2
        set_motor_B_pwm(50, 'forward')
    elif 64 <= joy_ly < 96:  # Segment 3
        set_motor_B_pwm(10, 'forward')
    elif 96 <= joy_ly < 128:  # Segment 4
        set_motor_B_pwm(0, 'coast') # Consider this as neutral or coast
    elif 128 <= joy_ly < 160:  # Segment 5
        set_motor_B_pwm(0, 'coast') # Consider this as neutral or coast
    elif 160 <= joy_ly < 192:  # Segment 6
        set_motor_B_pwm(10, 'reverse')
    elif 192 <= joy_ly < 224:  # Segment 7
        set_motor_B_pwm(50, 'reverse')
    elif 224 <= joy_ly <= 255:  # Segment 8
        set_motor_B_pwm(50, 'reverse')

In [9]:
#Right(A)-motor_set
def control_motor_based_on_joy_ry(joy_ry):
    # Control motor based on joy_ly value divided into 8 segments
    if 0 <= joy_ry < 32:  # Segment 1
        set_motor_A_pwm(50, 'forward')
    elif 32 <= joy_ry < 64:  # Segment 2
        set_motor_A_pwm(50, 'forward')
    elif 64 <= joy_ry < 96:  # Segment 3
        set_motor_A_pwm(10, 'forward')
    elif 96 <= joy_ry < 128:  # Segment 4
        set_motor_A_pwm(0, 'coast') # Consider this as neutral or coast
    elif 128 <= joy_ry < 160:  # Segment 5
        set_motor_A_pwm(0, 'coast') # Consider this as neutral or coast
    elif 160 <= joy_ry < 192:  # Segment 6
        set_motor_A_pwm(10, 'reverse')
    elif 192 <= joy_ry < 224:  # Segment 7
        set_motor_A_pwm(50, 'reverse')
    elif 224 <= joy_ry <= 255:  # Segment 8
        set_motor_A_pwm(50, 'reverse')

In [10]:
def control_motor_based_on_arm_f(h_time):
    gpio_out.write(0x27,mask) #arm_forward
    time.sleep(h_time)  # Wait
    gpio_out.write(0x07,mask) #arm_free

In [11]:
def control_motor_based_on_arm_r(h_time):
    gpio_out.write(0x47,mask) #arm_reverse
    time.sleep(h_time)  # Wait
    gpio_out.write(0x07,mask) #arm_free

In [ ]:
from inputs import get_gamepad

# Loop to monitor joystick input and control the motor based on it
try:
    while True:
        events = get_gamepad()
        for event in events:
            print(event.state)
            print(event.code)
            if event.ev_type == 'Absolute' and event.code == 'ABS_Y':  #Left(B)-motor
                print(event.state)
                print(event.code)
                joy_ly = int(event.state)
                control_motor_based_on_joy_ly(joy_ly)
            elif  event.ev_type == 'Absolute' and event.code == 'ABS_Z': #Right(A)-motor
                print(event.state)
                print(event.code)
                joy_ry = int(event.state)
                control_motor_based_on_joy_ry(joy_ry)
            elif  event.code == 'BTN_C':
                print(event.code)
                control_motor_based_on_arm_f(0.2)
            elif  event.code == 'BTN_EAST':
                print(event.code)
                control_motor_based_on_arm_r(0.2)
                
except KeyboardInterrupt:
    gpio_out.write(0x00,mask) #Blue_led_off + 2*motor_driver_disenable
    print("Exiting program.")

589830
MSC_SCAN
1
BTN_Z
0
SYN_REPORT
589830
MSC_SCAN
0
BTN_Z
0
SYN_REPORT
589830
MSC_SCAN
1
BTN_Z
0
SYN_REPORT
589830
MSC_SCAN
0
BTN_Z
0
SYN_REPORT
589829
MSC_SCAN
1
BTN_WEST
0
SYN_REPORT
589829
MSC_SCAN
0
BTN_WEST
0
SYN_REPORT
